# With Efficient net, I am able to get 98% Accuracy.
# Prior to this,I Tried to classify wihtout training network, just by using image net weights can get only 50% accuracy.
# By Training all layers in the network I am able to ge 98 % accurcy.
# Instead of Dense layer at the end,used Convolution + Dense Layer.


In [ ]:
!pip install -U efficientnet

In [ ]:
!pip install -q --upgrade wandb 

import wandb
print(wandb.__version__)
from wandb.keras import WandbCallback



In [ ]:
#wandb.login()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import StratifiedKFold
import tensorflow_addons as tfa
from sklearn.utils import class_weight
import os 

import efficientnet.keras as efn
from sklearn.metrics import confusion_matrix

import glob

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# How to enable data generator for TPU usage
Step1- to use TPU : https://www.kaggle.com/product-feedback/163416
https://www.kaggle.com/manjunathns/tensorflow-tpu-seti-efficientnet-train/edit

In [ ]:
AUTO_ENCODER = True
EFFNET = False

train_csv = "../input/seti-breakthrough-listen/train_labels.csv"
train_df_master = pd.read_csv( train_csv)
train_df_master["path"] = train_df_master["id"].apply( lambda x: "../input/seti-breakthrough-listen/train/"+ str(x[0]) +"/"+x +".npy" )
train_df_master.head()


# Lets Visualize few numpy files

In [ ]:
def create_heat_map( file_name,target  ):
    data = np.load( file_name  )
    plt.figure( figsize =( 16,8), dpi = 80 )
    for i in range( 0, 6):
        plt.subplot( 1, 6, i +1)
        sns.heatmap( data [ i ] )
        plt.suptitle ( file_name +" = "+ str ( target) )
    plt.show()
#create_heat_map ( train_df_master["path"].iloc[np.random.randint (1000) ]  )

In [ ]:
for i in range ( 0,4 ):
    if i % 2 == 0 & False :
        shape = train_df_master[train_df_master["target"] == 0 ].shape[0]
        file_name = train_df_master["path"].iloc[ shape]
        target = 0
    else:
        shape = train_df_master[train_df_master["target"] == 1 ].shape[0]
        file_name = train_df_master["path"].iloc[ shape ]
        target = 1
        
    create_heat_map( file_name, target  )
        

In [ ]:
fig = plt.figure( figsize=(10,5), dpi = 80 ) 
sns.countplot( train_df_master['target'])
plt.show()

*  Objetive is to identify anomally, and we have 6 channels.
*  In that channels 1,3,5 are man made & channel 0,2,4 are from man made + outer space.
*  Good part is, any channel 0,2,4 might have signal from Extra terestrial [ET]orall channels .
*  So for auto Encoder instead of taking entire image  -->not working ,asimgaedimension + network size too big tohandle 
     1. We can start training Auto Encoder using channel bychannel
     2. We could use all channels from target 0.

*  This way we can reduce the complexity of model to very small. as we are not lookign at image in3 Dimention. Rather we are going look at it only in 2 Dimension
* Same method will be applied by using any latest Pretrained models 

In [ ]:
## perform over sample 
if True:
    
    anamolly_df  = train_df_master[ train_df_master["target"] == 1]
    train_df_master = pd.concat (  [ train_df_master,anamolly_df,anamolly_df,anamolly_df ])
    train_df_master.shape

In [ ]:
fig = plt.figure( figsize=(10,5), dpi = 80 ) 
sns.countplot( train_df_master['target'])
plt.show()

In [ ]:
CFG= {
    
    "IMG_LENGTH" :  273,#256
    "IMG_WIDTH" : 256,
    "CHANNELS" : 3,
    "RANDOM_STATE" : 100,
    "BATCH_SIZE"  :100,
    "FOLDS" : 5,
    "LEARNING_RATE" : 0.1
}

CFG_ENCODE_DECODE= {
    
    "IMG_LENGTH" :  256,
    "IMG_WIDTH" : 256,
    "CHANNELS" : 3,
    "RANDOM_SATE" : 100,
    "BATCH_SIZE"  :50,
    "FOLDS" : 5,
    "LEARNING_RATE" : 0.1
}
    
    

1. Building data generator

In [ ]:

train_df,test_df = train_test_split ( train_df_master, train_size = 0.8, random_state= CFG["RANDOM_STATE"],shuffle = True,stratify = train_df_master["target"])
val_df, test_df = train_test_split ( test_df, test_size = 0.2 , random_state= CFG["RANDOM_STATE"],shuffle = True,stratify = test_df["target"])


print ("number of samples for train data set  = {} ".format(len ( train_df) ) )
print ("number of samples for test data set  = {} ".format(len ( test_df)))
print ("number of samples for validation data set  = {} ".format(len ( val_df ) ) )

In [ ]:
## Creating data generator which can work on Both TPU + GPU

def decode_numpy(  ):
    
    def read_image(file_name ):
        #print ( file_name)
        np_data =  tf.io.read_file ( file_name )
        np_data = tf.io.decode_raw( np_data, tf.float16 )
        np_data = tf.reshape( np_data[64:], (6, 273, 256 )) # (6, 273, 256 ) is data origional shape 
        #np_data = tf.concat ( (np_data[0],np_data[2],np_data[4]), axis = 0)
        #np_data = tf.stack([np_data[0],np_data[2],np_data[4]] ,axis = 2 ) 
        np_data = tf.stack([np_data[0],np_data[2],np_data[4]] ,axis = 2 )
        #np_data = tf.concat([np_data[0],np_data[2],np_data[4]] ,axis = -1  )
        #np_data = tf.stack([np_data,np_data,np_data] ,axis = 2 )
        np_data = tf.reshape( np_data, ( CFG["IMG_LENGTH"], CFG["IMG_WIDTH"], CFG["CHANNELS"] ))
        #np_data = tf.image.r(np_data)
        return np_data   #(819, 256)is output shape 
        
    def decode( file_name, target ):
        return read_image ( file_name ), target#tf.cast(target, tf.float32) 
    
    return decode

def data_augmentation( ):
    
    def add_augmentation( image ):
        
        image = tf.image.random_flip_up_down( image,seed= CFG["RANDOM_STATE"] )
        image = tf.image.random_flip_left_right( image ,seed= CFG["RANDOM_STATE"] )
        image = tf.image.random_contrast( image,lower =0.1,upper = 0.5 ,seed= CFG["RANDOM_STATE"] )
        image = tf.image.random_saturation( image,lower =5,upper = 7 ,seed= CFG["RANDOM_STATE"])
        #image = tf.image.random_crop( value = image , size = (3,3), seed=CFG["RANDOM_STATE"] )
        return image
    def call_add_augmentation( data, target ):
        return add_augmentation( data), target 
    
    return  call_add_augmentation

def datagenerator_rev_01(df,test = False ):
    file_list = df["path"].to_list() 
    target = df["target"].to_list() 
    decode_tf = decode_numpy()
    augment_fn = data_augmentation()
    
    datagen = tf.data.Dataset.from_tensor_slices( (file_list,target ))
    datagen = datagen.map( decode_tf ,num_parallel_calls= tf.data.AUTOTUNE )
    datagen = datagen.map(augment_fn, num_parallel_calls= tf.data.AUTOTUNE ) if not test else datagen
    datagen = datagen.repeat() if not test else datagen
    datagen = datagen.shuffle(1024) if not test else datagen
    datagen = datagen.batch(CFG["BATCH_SIZE"]).prefetch(tf.data.AUTOTUNE )
    return  datagen



In [ ]:
## Creating data generator which can work on Both TPU + GPU

def decode_numpy_auto_ecoder(  ):
    
    def read_image(file_name ):
        #print ( file_name)
        np_data =  tf.io.read_file ( file_name )
        np_data = tf.io.decode_raw( np_data, tf.float16 )
        np_data = tf.reshape( np_data[64:], (6, 273, 256 )) # (6, 273, 256 ) is data origional shape 
        np_data = tf.stack([np_data[1],np_data[3],np_data[5]] ,axis = 2 )
        np_data = np_data[np.random.randint(3) ]
        np_data = tf.stack([np_data,np_data,np_data] ,axis = 2 )
        np_data = tf.image.resize(np_data,( CFG_ENCODE_DECODE["IMG_LENGTH"], CFG_ENCODE_DECODE["IMG_WIDTH"] ) )
        return tf.data.Dataset.from_tensors( np_data[:,:,0:1] ) #(256, 256)is output shape 
        
    def decode( file_name ):
        return read_image ( file_name )
    
    return decode

def input_output():
    
    def auto_encode_input_output(data):
        return data,data
    
    return auto_encode_input_output


def Auto_Encoder_Data_Generator(df,test = False,channel =None ):
    file_list = df["path"].to_list() 
    target = df["target"].to_list() 
    decode_tf = decode_numpy_auto_ecoder()
    
    io_for_auto_encode =input_output()
    
    data_gen = tf.data.Dataset.from_tensor_slices( (file_list  ) )
    data_gen = data_gen.interleave(decode_tf,num_parallel_calls=tf.data.AUTOTUNE) 
    data_gen = data_gen.map( io_for_auto_encode,num_parallel_calls=tf.data.AUTOTUNE )
    data_gen = data_gen.shuffle( 30 ,seed = CFG_ENCODE_DECODE["RANDOM_SATE"], reshuffle_each_iteration = True ) if not test else data_gen
    data_gen = data_gen.batch( CFG_ENCODE_DECODE["BATCH_SIZE"]) if not test else data_gen
    data_gen = data_gen.prefetch( tf.data.AUTOTUNE )
    data_gen = data_gen.repeat() if not test else data_gen
    
    
    
    return  data_gen



In [ ]:
if False:
    
    def read_numpy_data( file_name_result ):

        #random_int = np.random.randint(3)
        #channel = [1,3,5]
        file_name=file_name_result
        data = np.load( file_name.numpy() ).astype( np.float16 )    
        data =np.vstack ( (data[0],data[2],data[4]) )
        #data = np.vstack( (data,data,data) )
        return  np.dstack ( [ data, data, data] ) 



    def resize_image( df_dict ):

        [image, ] =  tf.py_function( read_numpy_data,  [ df_dict["path"] ], [tf.float16] )
        image.set_shape( ( CFG.IMG_LENGTH,CFG.IMG_WIDTH, CFG.CHANNELS ) )
        image = tf.image.resize( image , ( CFG.IMG_LENGTH,CFG.IMG_WIDTH ) )
        label = df_dict["target"]
        label = tf.cast( label, tf.int16 )

        return  image, label 

    def data_augmentation( ):

        def add_augmentation( image ):

            #image = tf.image.random_flip_up_down( image )
            #image = tf.image.random_flip_up_down( image )
            #image = tf.image.random_contrast( image )
            return image
        def call_add_augmentation( data, target ):
            return add_augmentation( data), target 

        return  call_add_augmentation


    def create_tf_dataset( dataframe, test =False  ):

        augment_fn = data_augmentation()

        tf_dataset = tf.data.Dataset.from_tensor_slices ( dict ( dataframe[ ["path","target"] ] ) ) 
        
        if test :
        
            tf_dataset = ( tf_dataset
                          .map( resize_image,num_parallel_calls= tf.data.AUTOTUNE)
                          .batch( CFG.BATCH_SIZE )
                          .prefetch( tf.data.AUTOTUNE)
                         )
        else:
            tf_dataset = ( tf_dataset
                          .shuffle (1024)
                          .map( resize_image,num_parallel_calls= tf.data.AUTOTUNE)
                          .batch( CFG.BATCH_SIZE )
                          .prefetch( tf.data.AUTOTUNE)
                         )
        tf_dataset = tf_dataset.map (augment_fn,num_parallel_calls= tf.data.AUTOTUNE )

        return  tf_dataset  

    train_data_gen = create_tf_dataset( train_df ) 
    val_data_gen = create_tf_dataset( val_df ) 
    test_data_gen = create_tf_dataset ( test_df, test = True ) 

In [ ]:
def build_lrfn(lr_start=0.00001, lr_max=0.000075, lr_min=0.000001, lr_rampup_epochs=20, lr_sustain_epochs=0, lr_exp_decay=.8):
    lr_max = lr_max #* strategy.num_replicas_in_sync
    
    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_exp_decay ** (epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
        return lr
    
    return lrfn

In [ ]:
def get_model():
    
    
    input_mo = tf.keras.layers.Input(shape= (  CFG["IMG_LENGTH"], CFG["IMG_WIDTH"], CFG["CHANNELS"] ) )

    efff_net =tf.keras.applications.EfficientNetB0(include_top = False, 
                                                   weights ="imagenet" , 
                                                   input_shape = ( CFG["IMG_LENGTH"], CFG["IMG_WIDTH"], CFG["CHANNELS"]) ,
                                                   input_tensor = input_mo,
                                                   classes=2,
                                                   pooling = True,
                                                   #classifier_activation='softmax',
                                                   drop_connect_rate= 0.1
                                                  )

    for layer in  efff_net.layers  :
        #if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True
        

    model = tf.keras.Sequential( [ efff_net,
            tf.keras.layers.GlobalMaxPool2D(),
            #tf.keras.layers.Dropout(0.25 ),

            #tf.keras.layers.Dense(600, activation='relu'),
            #tf.keras.layers.Flatten(),
            #tf.keras.layers.Dropout(0.2 ),
            tf.keras.layers.Dense(32, activation='relu'),
            
            tf.keras.layers.Dense(1, activation='sigmoid')
            
            ])    
    
    
    #model.summary()
    
    #model.compile(optimizer, 
    #              loss=tfa.losses.SigmoidFocalCrossEntropy(), 
    #              metrics=[tf.keras.metrics.AUC(curve='ROC')])
    
    return ( model )

In [ ]:
def get_model_effnet_07():
    
    
    #input_ = layers.Input((3*273,256,3))
    input_ = tf.keras.layers.Input((273,256,3))
    
    #x = efn.EfficientNetB0(input_shape=(3*273,256,3),weights='noisy-student',include_top=False)(input_)
    x = efn.EfficientNetB7(input_shape=(273,256,3),weights='noisy-student',include_top=False)(input_)
    #x = layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.GlobalMaxPool2D()(x)
    x = tf.keras.layers.Dense(32)(x)
    x = tf.keras.layers.Activation("relu")(x)
    
    x = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    
    model = tf.keras.models.Model(inputs=input_, outputs=x)   
    
    
    #model.summary()
    
    #model.compile(optimizer, 
    #              loss=tfa.losses.SigmoidFocalCrossEntropy(), 
    #              metrics=[tf.keras.metrics.AUC(curve='ROC')])
    
    return ( model )

In [ ]:
#x = efn.EfficientNetB0(input_shape=(273,256,3),weights='noisy-student',include_top=False)(input_)


def get_model_noisy_student_weights():
    
    
    input_mo = tf.keras.layers.Input(shape= (  CFG["IMG_LENGTH"], CFG["IMG_WIDTH"], CFG["CHANNELS"] ) )

    efff_net =efn.EfficientNetB0(include_top = False, 
                                                   weights ="noisy-student" , 
                                                   input_shape = ( CFG["IMG_LENGTH"], CFG["IMG_WIDTH"], CFG["CHANNELS"]) ,
                                                   input_tensor = input_mo,
                                                   classes=2,
                                                   pooling = True,
                                                   #classifier_activation='softmax',
                                                   #drop_connect_rate= 0.1
                                                  )

    for layer in  efff_net.layers  :
        #if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False
        

    model = tf.keras.Sequential( [#tf.keras.layers.GaussianNoise( stddev=0.3 ),
                                  efff_net,
                                  tf.keras.layers.Flatten(),
                                  #tf.keras.layers.GlobalAveragePooling2D(),
            
            #tf.keras.layers.Dropout(0.25 ),
            tf.keras.layers.Dense(1000, activation='relu'),
            tf.keras.layers.Dropout(0.1 ),
            tf.keras.layers.Dense(100, activation='relu'),
            tf.keras.layers.Dropout(0.1 ),
            tf.keras.layers.Dense(32, activation='relu'),
            
            tf.keras.layers.Dense(1, activation='sigmoid')
            
            ])    
    
    
    #x = layers.GlobalAveragePooling2D()(x)
    
    #x = layers.Dense(32)(x)
    #x = layers.Activation("relu")(x)
    
    #x = layers.Dense(1, activation="sigmoid")(x)
    
    
    return ( model )

In [ ]:
#model_1 = get_model_noisy_student_weights()
#model_1.summary()

In [ ]:
if False:
    

    CFG["BATCH_SIZE"] =100

    lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(  patience=3,
                                                        min_lr= 0.000001,
                                                        monitor='val_loss', 
                                                        factor=0.6, 
                                                        verbose=1,
                                                        mode='auto', 
                                                       )

    train_df_2 = train_df[["path","target"]].iloc[:30000]
    val_df_2 = val_df[["path","target"]].iloc[:5000]

    CFG['model_name'] = 'efficientnetb0-folds'
    CFG['group'] = 'K-Fold-EnetB0'
    CFG['run_name'] = 'baseline-k-fold'
    model = get_model()

    class_weight_for_train = {0:1, 1: (train_df_2[train_df_2["target"]==0].shape[0]/train_df_2[train_df_2["target"]==1].shape[0]) }
    CFG["TRAIN_STEPS"] = int ( train_df_2.shape[0]/CFG["BATCH_SIZE"] ) + (1 if train_df_2.shape[0]% CFG["BATCH_SIZE"] != 0 else 0)
    CFG["VAL_STEPS"] = int ( val_df_2.shape[0]/CFG["BATCH_SIZE"] ) + (1 if val_df_2.shape[0]% CFG["BATCH_SIZE"] != 0 else 0)

    train_data_gen = datagenerator_rev_01( train_df_2 ) 
    val_data_gen = datagenerator_rev_01( val_df_2 ) 



    #model = get_model()

    model.build( ( None, CFG["IMG_LENGTH"], CFG["IMG_WIDTH"], CFG["CHANNELS"]) )
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0006000000284984708)#0.001) #tf.keras.optimizers.Adam(learning_rate=0.1)
    model.compile( optimizer= optimizer,loss="binary_crossentropy", metrics=[tf.keras.metrics.AUC(curve='ROC')])

    checkpoint = tf.keras.callbacks.ModelCheckpoint( f'model{1}.h5', save_best_only=True, monitor='val_loss', mode='min')


    model_history = model.fit_generator(train_data_gen,
                                        #class_weight= class_weight_for_train,
                                         steps_per_epoch= CFG["TRAIN_STEPS"], 
                                         epochs =5, 
                                         validation_data= val_data_gen,
                                         validation_steps = CFG["VAL_STEPS"],
                                         callbacks=[ checkpoint,lr_reducer]
                                       )

    # Initialize W&B run
    run = wandb.init(project='kaggle-seti', 
                     config=CFG,
                     group=CFG['group'], 
                     job_type='train',
                     name=CFG['run_name'])

    # Evaluate
    loss, auc = model.evaluate( val_data_gen, steps = CFG["VAL_STEPS"])
    wandb.log({'Val AUC-ROC': auc})

    # Save model
    model_name = CFG['model_name']
    MODEL_PATH= f'models/{model_name}'
    os.makedirs(MODEL_PATH, exist_ok=True)
    count_models = len(os.listdir(MODEL_PATH))

    model.save(f'{MODEL_PATH}/{model_name}_{count_models}.h5')

    run.finish()



In [ ]:
if False:
    TEST_STEPS = int ( test_df.shape[0]/CFG["BATCH_SIZE"] ) + (1 if test_df.shape[0]% CFG["BATCH_SIZE"] != 0 else 0)
    test_data_gen = datagenerator_rev_01 ( test_df[["path","target"]],True )    
    prediction = model.predict_classes( test_data_gen, batch_size = CFG["BATCH_SIZE"] )

In [ ]:
if False:
    test_predict_df = test_df
    test_predict_df["prediction"]= prediction
    test_predict_df.head()

In [ ]:
if False: confusion_matrix( test_predict_df["target"], test_predict_df["prediction"])

In [ ]:
def create_model():
    #input_ = layers.Input((3*273,256,3))
    input_ = tf.keras.layers.Input((273,256,3))
    
    #x = efn.EfficientNetB0(input_shape=(3*273,256,3),weights='noisy-student',include_top=False)(input_)
    x = efn.EfficientNetB0(input_shape=(273,256,3),weights='noisy-student',include_top=False)(input_)
    #x = layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.GlobalMaxPool2D()(x)
    x = tf.keras.layers.Dense(32)(x)
    x = tf.keras.layers.Activation("relu")(x)
    
    x = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    
    model = tf.keras.models.Model(inputs=input_, outputs=x)
    
    return model

In [ ]:
## Mode building with Efficient net with pre trained model 

if EFFNET :
    
    nosy_eff_net = create_model ()
    #nosy_eff_net.build( ( None, CFG["IMG_LENGTH"], CFG["IMG_WIDTH"], CFG["CHANNELS"]) )
    nosy_eff_net.load_weights("../input/seti-tpu-rev-02/model_TPU_REV_02.h5")

    
    CFG["BATCH_SIZE"] =75

    lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(  patience=1,
                                                        min_lr= 0.0000001,
                                                        monitor='val_loss', 
                                                        factor=0.6, 
                                                        verbose=1,
                                                        mode='auto', 
                                                       )

    train_df_2 = train_df[["path","target"]]#.iloc[:30000]
    val_df_2 = val_df[["path","target"]]#.iloc[:5000]

    CFG['model_name'] = 'efficientnetb0-folds_nosy_oversample'
    CFG['group'] = 'K-Fold-EnetB0_nosy'
    CFG['run_name'] = 'baseline-k-fold_nosy'


    #class_weight_for_train = {0:1, 1: (train_df_2[train_df_2["target"]==0].shape[0]/train_df_2[train_df_2["target"]==1].shape[0]) }
    CFG["TRAIN_STEPS"] = int ( train_df_2.shape[0]/CFG["BATCH_SIZE"] ) + (1 if train_df_2.shape[0]% CFG["BATCH_SIZE"] != 0 else 0)
    CFG["VAL_STEPS"] = int ( val_df_2.shape[0]/CFG["BATCH_SIZE"] ) + (1 if val_df_2.shape[0]% CFG["BATCH_SIZE"] != 0 else 0)

    train_data_gen = datagenerator_rev_01( train_df_2 ) 
    val_data_gen = datagenerator_rev_01( val_df_2 ) 




    #0.0006000000284984708
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0003600000170990825  ) #tf.keras.optimizers.Adam(learning_rate=0.1)
    nosy_eff_net.compile( optimizer= optimizer,loss="binary_crossentropy", metrics=[tf.keras.metrics.AUC(curve='ROC')])

    checkpoint = tf.keras.callbacks.ModelCheckpoint( f'model{1}.h5', save_best_only=True, monitor='val_loss', mode='min')


    model_history = nosy_eff_net.fit_generator(train_data_gen,
                                        #class_weight= class_weight_for_train,
                                         steps_per_epoch= CFG["TRAIN_STEPS"], 
                                         epochs =5, 
                                         validation_data= val_data_gen,
                                         validation_steps = CFG["VAL_STEPS"],
                                         callbacks=[ checkpoint,lr_reducer]
                                       )
    nosy_eff_net.save("./nosy_eff_net_with_guassian_noise_over_sample_part_2.h5")
    # Initialize W&B run
    run = wandb.init(project='kaggle-seti', 
                     config=CFG,
                     group=CFG['group'], 

                     job_type='train',
                     name=CFG['run_name'])

    # Evaluate
    loss, auc = nosy_eff_net.evaluate( val_data_gen, steps = CFG["VAL_STEPS"])
    wandb.log({'Val AUC-ROC': auc})

    # Save model
    model_name = CFG['model_name']
    MODEL_PATH= f'models/{model_name}'
    os.makedirs(MODEL_PATH, exist_ok=True)
    count_models = len(os.listdir(MODEL_PATH))

    nosy_eff_net.save(f'{MODEL_PATH}/{model_name}_{count_models}.h5')

    run.finish()

In [ ]:
if EFFNET :
    nosy_eff_net = get_model_effnet_07()
    nosy_eff_net.load_weights("../input/setitpurev03/model_TPU_REV_03.h5")

In [ ]:
if EFFNET :
    submission_df = pd.read_csv("../input/seti-breakthrough-listen/sample_submission.csv")
    submission_df["path"] = submission_df["id"].apply( lambda x: "../input/seti-breakthrough-listen/test/"+ str(x[0]) +"/"+x +".npy" )
    submission_df["target"] = [1]*submission_df.shape[0]
    submission_df.head()

In [ ]:
if EFFNET : submission_df.shape

In [ ]:
if EFFNET : CFG["BATCH_SIZE"] = 50

In [ ]:
 if EFFNET :
    fial_submission_data_gen = datagenerator_rev_01 ( submission_df[["path","target"]], True ) 
    final_prediction = nosy_eff_net.predict( fial_submission_data_gen, batch_size = CFG["BATCH_SIZE"] )
    submission_df["target"]= final_prediction


In [ ]:
if EFFNET : submission_df.head(10)

In [ ]:
if EFFNET :
    final_submission = submission_df[["id","target"]]
    final_submission.head()

In [ ]:
if EFFNET : final_submission.to_csv("./sample_submission.csv",index = False)

In [ ]:
if EFFNET : test_data_gen = datagenerator_rev_01( test_df[["path","target"]], True ) 

In [ ]:
#nosy_eff_net = get_model_noisy_student_weights ()
#nosy_eff_net.load_weights("../input/train-model-weight-for-seti-anomally-detection/PRETRAINED_MODEL_WEIGHTS.h5")

In [ ]:

if EFFNET : prediction_df = nosy_eff_net.predict_classes(test_data_gen ) 

In [ ]:
if EFFNET : test_df["prediction"] = prediction_df

In [ ]:
if EFFNET : confusion_matrix ( test_df["target"],test_df["prediction"] )

#import neural_structured_learning as nsl
CFG["BATCH_SIZE"] =100

lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(  patience=1,
                                                    min_lr= 0.000001,
                                                    monitor='val_loss', 
                                                    factor=0.5, 
                                                    verbose=1,
                                                    mode='auto', 
                                                   )

#lr_reducer = build_lrfn()

KFOLD = StratifiedKFold( n_splits= CFG["FOLDS"],shuffle = True, random_state= CFG["RANDOM_STATE"] )    
kfold_history = []
CFG['model_name'] = 'efficientnetb0-folds'
CFG['group'] = 'K-Fold-EnetB0'
CFG['run_name'] = 'baseline-k-fold'
model_1 = get_model()

for i , (train_index,val_index) in enumerate( KFOLD.split( train_df,train_df["target"] ) ):
    
    
    train = train_df.iloc[train_index]
    val = train_df.iloc[val_index]
    
    class_weight_for_train = {0:1, 1: (train[train["target"]==0].shape[0]/train[train["target"]==1].shape[0]) }
    
    print ( "class weight =={}",class_weight_for_train )
    print ( "Curent fold =={}".format( i))
    
    CFG["TRAIN_STEPS"] = int ( train.shape[0]/CFG["BATCH_SIZE"] ) + (1 if train.shape[0]% CFG["BATCH_SIZE"] != 0 else 0)
    CFG["VAL_STEPS"] = int ( val.shape[0]/CFG["BATCH_SIZE"] ) + (1 if val.shape[0]% CFG["BATCH_SIZE"] != 0 else 0)


    train_data_gen = datagenerator_rev_01( train[["path","target"]] ) 
    val_data_gen = datagenerator_rev_01( val[["path","target"]] ) 
    #test_data_gen = datagenerator_rev_01 ( test_df[["path","target"]])    

    
    model_1 = get_model()
    
    
    model_1.build( ( None, CFG["IMG_LENGTH"], CFG["IMG_WIDTH"], CFG["CHANNELS"]) )
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.05) #tf.keras.optimizers.Adam(learning_rate=0.1)
    model_1.compile( optimizer= optimizer,loss="binary_crossentropy", metrics=[tf.keras.metrics.AUC(curve='ROC')])


    checkpoint = tf.keras.callbacks.ModelCheckpoint( f'model{i}.h5', save_best_only=True, monitor='val_loss', mode='min')


    model_history = model_1.fit_generator(train_data_gen,
                                                  class_weight= class_weight_for_train,#{0: 0.5514850705113055, 1: 5.355776587605202},
                                                 steps_per_epoch= CFG["TRAIN_STEPS"], 
                                                 epochs =5, 
                                                 validation_data= val_data_gen,
                                                 validation_steps = CFG["VAL_STEPS"],
                                                callbacks=[ checkpoint,lr_reducer#tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1),
                                                              #tf.keras.callbacks.ModelCheckpoint( "../ZIP_MODEL/model2.h5", 
                                                               #                              monitor='loss', 
                                                               #                              verbose=0,
                                                               #                              save_best_only=True, 
                                                               #                              save_weights_only=True,
                                                               #                              mode='auto', 
                                                               #                              save_freq='epoch',
                                                                                             #monitor='val_loss', 
                                                                                             #mode='min'
                                                                #
                                                                #                            )
                                      ] )
    
    # Initialize W&B run
    run = wandb.init(project='kaggle-seti', 
                     config=CFG,
                     group=CFG['group'], 
                     job_type='train',
                     name=CFG['run_name'])
    
    # Evaluate
    loss, auc = model_1.evaluate( val_data_gen, steps = CFG["VAL_STEPS"])
    wandb.log({'Val AUC-ROC': auc})
    
    # Save model
    model_name = CFG['model_name']
    MODEL_PATH = f'models/{model_name}'
    os.makedirs(MODEL_PATH, exist_ok=True)
    count_models = len(os.listdir(MODEL_PATH))
    
    model_1.save(f'{MODEL_PATH}/{model_name}_{count_models}.h5')

    # Get Prediction on validation set
    #_oof_df = get_predictions(model, validloader, valid_df)
    #oof_df = pd.concat([oof_df, _oof_df])
    
    run.finish()
    
    #kfold_history.append( model_1 )

In [ ]:
encoder_decoder_model = tf.keras.Sequential(
            [ tf.keras.layers.Input( shape=  ( CFG_ENCODE_DECODE["IMG_LENGTH"],CFG_ENCODE_DECODE["IMG_WIDTH"] ,1 ), name= "encoder_input_layer" ) ,
              tf.keras.layers.Conv2D(filters = 256, kernel_size = (4,4), strides = 2,padding = "valid",activation ="relu",name ="encoder_layer_01"),
              tf.keras.layers.Conv2D(filters = 128, kernel_size = (4,4), strides = 2,padding = "valid",activation ="relu",name ="encoder_layer_02"),
              tf.keras.layers.Conv2D(filters = 64, kernel_size =  (4,4), strides = 2,padding = "valid",activation ="relu",name ="encoder_layer_03") ,
              tf.keras.layers.Conv2D(filters = 32, kernel_size =  (4,4), strides = 2,padding = "valid",activation ="relu",name ="encoder_layer_04") ,
              tf.keras.layers.Conv2D(filters = 16, kernel_size =  (4,4), strides = 2,padding = "valid",activation ="relu",name ="encoder_layer_05"),
              #tf.keras.layers.Conv2D(filters = 8, kernel_size =  (4,4), strides = 2,padding = "valid",name ="encoder_layer_65"),
              #tf.keras.layers.Conv1D(filters = 8, kernel_size = (4,4), strides = 3,padding = "same",activation ="relu"),

              tf.keras.layers.Conv2DTranspose(filters = 16, kernel_size = (4,4), strides = 2,padding = "valid",activation ="relu",name ="decoder_layer_01"),
              tf.keras.layers.Conv2DTranspose(filters = 32, kernel_size = (4,4), strides = 2,padding = "valid",activation ="relu",name ="decoder_layer_02"),
              tf.keras.layers.Conv2DTranspose(filters = 64, kernel_size = (4,4), strides = 2,padding = "valid",activation ="relu",name ="decoder_layer_03"),
              tf.keras.layers.Conv2DTranspose(filters = 128, kernel_size = (4,4), strides = 2,padding = "valid",activation ="relu",name ="decoder_layer_04"),
              tf.keras.layers.Conv2DTranspose(filters = 256, kernel_size = (4,4), strides = 2,padding = "valid",activation ="relu",name ="decoder_layer_05"),
              #tf.keras.layers.Conv2DTranspose(filters = 512, kernel_size = (4,4), strides = 2,padding = "valid",name ="decoder_layer_06"),
              tf.keras.layers.Conv2DTranspose(filters = 1, kernel_size = 3, strides = 1,padding = "valid",activation ="relu",name ="output_layer")
            ]
                    )
encoder_decoder_model.summary()

In [ ]:
train_df_2= train_df.iloc[:30000]
val_df_2 = val_df.iloc[:4000]
train_data_gen = Auto_Encoder_Data_Generator( train_df_2 )
val_data_gen = Auto_Encoder_Data_Generator( val_df_2 )
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(  patience=4,
                                                        min_lr= 0.0000001,
                                                        monitor='val_loss', 
                                                        factor=0.6, 
                                                        verbose=1,
                                                        mode='auto', 
                                                       )

CFG_ENCODE_DECODE["TRAIN_STEPS"] = int ( train_df_2.shape[0]/CFG_ENCODE_DECODE["BATCH_SIZE"] ) + (1 if train_df_2.shape[0] % CFG_ENCODE_DECODE["BATCH_SIZE"] != 0 else 0)
CFG_ENCODE_DECODE["VAL_STEPS"] = int ( val_df_2.shape[0]/CFG_ENCODE_DECODE["BATCH_SIZE"] ) + (1 if val_df_2.shape[0]% CFG_ENCODE_DECODE["BATCH_SIZE"] != 0 else 0)

optimizer = tf.keras.optimizers.Adam(learning_rate= 0.01  )

encoder_decoder_model.compile( optimizer= optimizer,loss="mse", metrics=[tf.keras.metrics.MeanSquaredError()])

checkpoint = tf.keras.callbacks.ModelCheckpoint( f'model{1}.h5', save_best_only=True, monitor='val_loss', mode='min')

encoder_decoder_model.fit(train_data_gen,
                        #class_weight= class_weight_for_train,
                         steps_per_epoch= CFG_ENCODE_DECODE["TRAIN_STEPS"], 
                         epochs =5, 
                         validation_data= val_data_gen,
                         validation_steps = CFG_ENCODE_DECODE["VAL_STEPS"],
                         callbacks=[ lr_reducer]
                       )
    
encoder_decoder_model.save("./encoder_decoder.h5" )


In [ ]:
CFG["BATCH_SIZE"]